In [1]:
import argparse
import logging
import sys
import time

from tf_pose import common
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
class MyArgs():
    def __init__(self):
        self.image = './images/p1.jpg'
        self.model = 'mobilenet_thin'
        self.resize = '0x0'
        self.resize_out_ratio = 4.0
args = MyArgs()
w, h = model_wh(args.resize)
if w == 0 or h == 0:
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(432, 368))
else:
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))

[2020-08-21 06:28:06,911] [TfPoseEstimator] [INFO] loading graph from /home/jovyan/work/fall-detection/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=432x368)
2020-08-21 06:28:06,911 INFO loading graph from /home/jovyan/work/fall-detection/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=432x368)


TfPoseEstimator/Openpose/concat_stage7/axis
TfPoseEstimator/Mconv7_stage6_L2/biases
TfPoseEstimator/Mconv7_stage6_L2/weights
TfPoseEstimator/Mconv6_stage6_L2/biases
TfPoseEstimator/Mconv6_stage6_L2/weights
TfPoseEstimator/Mconv5_stage6_L2/biases
TfPoseEstimator/Mconv5_stage6_L2/weights
TfPoseEstimator/Mconv4_stage6_L2/biases
TfPoseEstimator/Mconv4_stage6_L2/weights
TfPoseEstimator/Mconv3_stage6_L2/biases
TfPoseEstimator/Mconv3_stage6_L2/weights
TfPoseEstimator/Mconv2_stage6_L2/biases
TfPoseEstimator/Mconv2_stage6_L2/weights
TfPoseEstimator/Mconv1_stage6_L2/biases
TfPoseEstimator/Mconv1_stage6_L2/weights
TfPoseEstimator/Mconv7_stage6_L1/biases
TfPoseEstimator/Mconv7_stage6_L1/weights
TfPoseEstimator/Mconv6_stage6_L1/biases
TfPoseEstimator/Mconv6_stage6_L1/weights
TfPoseEstimator/Mconv5_stage6_L1/biases
TfPoseEstimator/Mconv5_stage6_L1/weights
TfPoseEstimator/Mconv4_stage6_L1/biases
TfPoseEstimator/Mconv4_stage6_L1/weights
TfPoseEstimator/Mconv3_stage6_L1/biases
TfPoseEstimator/Mconv3_st

In [3]:
def handleImage(image):
    global e
    try:
        humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=args.resize_out_ratio)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
        return humans
    except Exception as e:
        cv2.waitKey()
        return None

In [4]:
import os
import numpy as np
import csv
import pandas as pd
import functools
import operator

HUMAN_POINTS_NUMS = 17

def listFlat(l):
    return functools.reduce(operator.iconcat, l, [])

def handleHumanData(h):
    tmp_data = []
    for i in range(0, HUMAN_POINTS_NUMS + 1):
        if i in h.body_parts.keys():
            p = h.body_parts[i]
            x = p.x
            y = p.y
            score = p.score
        else:
            x = np.nan
            y = np.nan
            score = np.nan
        tmp_data.append({ 'x': x, 'y': y, 'score': score })
    df = pd.DataFrame(tmp_data)
    df['x'] = df['x'] - df['x'].mean()
    df['y'] = df['y'] - df['y'].mean()
    df['x'].fillna(value=0, inplace=True)
    df['y'].fillna(value=0, inplace=True)
    df['score'].fillna(value=df['score'].mean(), inplace=True)
    return listFlat(df.values)

In [5]:
import requests

def lineNotifyMessage(msg):
    token = '7QI2XKUNR95IAB1SiKSLCDNaqJIrZoz0Kx1zH8HWH2T'
    headers = {
        "Authorization": "Bearer " + token, 
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    payload = {'message': msg}
    r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    return r.status_code

In [6]:
from tensorflow import keras
model = keras.models.load_model('mdata/model/model_1')

In [7]:
import time

last_img = None
def start(video_path):
    global model
    global last_img
    TARGET_FALL = 0
    
    cam = cv2.VideoCapture(video_path)
    
    FPS_FIX = 5 #fps15，fps_fix取3，一秒5張
    DETECTION_THRESHOLD = 5 # 連續偵測到幾次才判斷跌倒
    INTERVAL_MINIMUM = 30 # 至少隔幾秒才能再發送一次通知
    SAVE_PATH = './mdata/detection_images/t' + str(time.time()).split('.')[0]
    
    os.mkdir(SAVE_PATH)
    
    start_flag = False
    ret_val = True
    count = -1
    detection_count = 0
    fall_img_count = 0
    
    dim = (720, 405)
    jpeg_flag = [int(cv2.IMWRITE_JPEG_QUALITY), 50]
    
    last_time = time.time()
    
    while ret_val:
        ret_val, img = cam.read()
        count = count + 1
        if count % FPS_FIX != 0:
            continue
        
        if not start_flag:
            if img is not None:
                print('Original image: (%d, %d)' % (img.shape[1], img.shape[0]))
            start_flag = True
        if img is not None:
            # image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            # result, encimg = cv2.imencode('.jpg', image, jpeg_flag)
            # decimg = cv2.imdecode(encimg, 1)
            img = cv2.GaussianBlur(img, (31, 31), 0)
            # (T, img) = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
            
            has_fall = False
            last_img = img
            humans = handleImage(img)
            for i, h in enumerate(humans):
                if len(h.body_parts.keys()) > HUMAN_POINTS_NUMS/2:
                    data = handleHumanData(h)
                    data = np.array(data)
                    data = data.reshape((1, 54))
                    res = model.predict_classes(data)
                    if res[0] == TARGET_FALL:
                        detection_count = detection_count + 1
                        has_fall = True
                        cur_time = time.time()
                        if (detection_count >= DETECTION_THRESHOLD) && (cur_time - last_time > INTERVAL_MINIMUM):
                            print('Fall...')
                            lineNotifyMessage('偵測到有人跌倒...')
                            cv2.imwrite((SAVE_PATH + "/img_%d.jpg") % fall_img_count, img)
                            fall_img_count = fall_img_count + 1
                            detection_count = 0
                            last_time = cur_time
                            # return
                    # return
            if not has_fall: # 要連續偵測到才算，而非累加
                detection_count = 0
        else:
            print('frame is empty...')
        
        if cv2.waitKey(1) == 27:
            break

    cv2.destroyAllWindows()

In [8]:
start('rtsp://120.105.129.229:1554/stream2')

Original image: (640, 480)
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...
Fall...


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(last_img, cv2.COLOR_BGR2RGB))

In [2]:
import requests
requests.post("https://fall-detection-line-notify.406.csie.nuu.edu.tw/api/notify")

<Response [504]>

In [3]:
def lineNotifyMessage(msg):
    token = 'crML93EU87PR9drj84tHI88cUvqijnt0Htxx7DxKI5M'
    headers = {
        "Authorization": "Bearer " + token, 
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    payload = {'message': msg}
    r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    return r.status_code

In [4]:
lineNotifyMessage("測試")

200